In [1]:
import os

db_user = "root"
db_password = "1532"
db_host = "localhost"
db_name = "classicmodels"
from langchain_community.utilities.sql_database import SQLDatabase
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


mysql
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']

CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmit

In [5]:
from langchain.chains import create_sql_query_chain
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="sqlcoder", temperature=0.8)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
write_query = create_sql_query_chain(llm, db)
query = write_query.invoke({"question": "give me all the payment detail with customer name"})
query

'SELECT customers.customerName, payments.paymentDate, payments.amount FROM customers JOIN payments ON customers.customerNumber = payments.customerNumber;'

In [ ]:
db.run(query)

In [63]:
x = db._execute('SELECT p.paymentDate, p.amount, c.customerName FROM payments p JOIN customers c ON p.customerNumber = c.customerNumber;')

In [64]:
x

[{'paymentDate': datetime.date(2004, 10, 19),
  'amount': Decimal('6066.78'),
  'customerName': 'Atelier graphique'},
 {'paymentDate': datetime.date(2003, 6, 5),
  'amount': Decimal('14571.44'),
  'customerName': 'Atelier graphique'},
 {'paymentDate': datetime.date(2004, 12, 18),
  'amount': Decimal('1676.14'),
  'customerName': 'Atelier graphique'},
 {'paymentDate': datetime.date(2004, 12, 17),
  'amount': Decimal('14191.12'),
  'customerName': 'Signal Gift Stores'},
 {'paymentDate': datetime.date(2003, 6, 6),
  'amount': Decimal('32641.98'),
  'customerName': 'Signal Gift Stores'},
 {'paymentDate': datetime.date(2004, 8, 20),
  'amount': Decimal('33347.88'),
  'customerName': 'Signal Gift Stores'},
 {'paymentDate': datetime.date(2003, 5, 20),
  'amount': Decimal('45864.03'),
  'customerName': 'Australian Collectors, Co.'},
 {'paymentDate': datetime.date(2004, 12, 15),
  'amount': Decimal('82261.22'),
  'customerName': 'Australian Collectors, Co.'},
 {'paymentDate': datetime.date(2003

In [65]:
for x1 in x:
    print(x1)

{'paymentDate': datetime.date(2004, 10, 19), 'amount': Decimal('6066.78'), 'customerName': 'Atelier graphique'}
{'paymentDate': datetime.date(2003, 6, 5), 'amount': Decimal('14571.44'), 'customerName': 'Atelier graphique'}
{'paymentDate': datetime.date(2004, 12, 18), 'amount': Decimal('1676.14'), 'customerName': 'Atelier graphique'}
{'paymentDate': datetime.date(2004, 12, 17), 'amount': Decimal('14191.12'), 'customerName': 'Signal Gift Stores'}
{'paymentDate': datetime.date(2003, 6, 6), 'amount': Decimal('32641.98'), 'customerName': 'Signal Gift Stores'}
{'paymentDate': datetime.date(2004, 8, 20), 'amount': Decimal('33347.88'), 'customerName': 'Signal Gift Stores'}
{'paymentDate': datetime.date(2003, 5, 20), 'amount': Decimal('45864.03'), 'customerName': 'Australian Collectors, Co.'}
{'paymentDate': datetime.date(2004, 12, 15), 'amount': Decimal('82261.22'), 'customerName': 'Australian Collectors, Co.'}
{'paymentDate': datetime.date(2003, 5, 31), 'amount': Decimal('7565.08'), 'customer

In [73]:
import pandas as pd
import datetime
from decimal import Decimal

# Preprocess data for DataFrame
processed_data = []
# for row in x:
#     processed_row = {key: (value.isoformat() if isinstance(value, datetime.date) else float(value) if isinstance(value, Decimal) else value) for key, value in row.items()}
#     processed_data.append(processed_row)

# # Create a DataFrame
df = pd.DataFrame(x)

# # Display the DataFrame as a table
print(df)

    paymentDate    amount          customerName
0    2004-10-19   6066.78     Atelier graphique
1    2003-06-05  14571.44     Atelier graphique
2    2004-12-18   1676.14     Atelier graphique
3    2004-12-17  14191.12    Signal Gift Stores
4    2003-06-06  32641.98    Signal Gift Stores
..          ...       ...                   ...
268  2003-12-26  59265.14  Diecast Collectables
269  2004-05-14   6276.60  Diecast Collectables
270  2005-05-25  30253.75     Kelly's Gift Shop
271  2003-07-16  32077.44     Kelly's Gift Shop
272  2004-12-31  52166.00     Kelly's Gift Shop

[273 rows x 3 columns]


In [48]:
x[0]

{'paymentDate': datetime.date(2004, 10, 19),
 'amount': Decimal('6066.78'),
 'customerName': 'Atelier graphique'}

In [79]:
# Sample data
data = [
    {'paymentDate': '2004-10-19', 'amount': '6066.78', 'customerName': 'Atelier graphique'},
    {'paymentDate': '2004-11-10', 'amount': '12000.00', 'customerName': 'Signal Gift Stores'},
    {'paymentDate': '2004-12-14', 'amount': '5200.50', 'customerName': 'Australian Collectors, Co.'}
]

# Generate HTML table
html = '<table border="1"><tr><th>Payment Date</th><th>Amount</th><th>Customer Name</th></tr>'
for row in data:
    html += '<tr>'
    html += f'<td>{row["paymentDate"]}</td>'
    html += f'<td>{row["amount"]}</td>'
    html += f'<td>{row["customerName"]}</td>'
    html += '</tr>'
html += '</table>'

# Print or save the HTML
print(html)


<table border="1"><tr><th>Payment Date</th><th>Amount</th><th>Customer Name</th></tr><tr><td>2004-10-19</td><td>6066.78</td><td>Atelier graphique</td></tr><tr><td>2004-11-10</td><td>12000.00</td><td>Signal Gift Stores</td></tr><tr><td>2004-12-14</td><td>5200.50</td><td>Australian Collectors, Co.</td></tr></table>


In [80]:
import csv
import datetime
from decimal import Decimal

# Assuming db._execute executes the SQL query and fetches the results as a list of dictionaries
x = db._execute('SELECT p.paymentDate, p.amount, c.customerName FROM payments p JOIN customers c ON p.customerNumber = c.customerNumber;')

# Extract column names from the keys of the first dictionary (assuming x is not empty)
column_names = x[0].keys()

# Preprocess data for CSV (convert datetime.date and Decimal to string)
# processed_data = []
# for row in x:
#     processed_row = {
#         key: (value.isoformat() if isinstance(value, datetime.date) else str(value) if isinstance(value, Decimal) else value)
#         for key, value in row.items()
#     }
#     # print(processed_row)
#     processed_data.append(processed_row)

# print(processed_data)
# Specify the CSV file name
csv_file_name = 'output.csv'

# Write the results to a CSV file
# with open(csv_file_name, mode='w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=column_names)
#     writer.writeheader()
#     writer.writerows(x)

# print(f'Data successfully written to {csv_file_name}')

Data successfully written to output.csv


In [15]:
x

'[(datetime.date(2004, 10, 19), Decimal(\'6066.78\'), \'Atelier graphique\'), (datetime.date(2003, 6, 5), Decimal(\'14571.44\'), \'Atelier graphique\'), (datetime.date(2004, 12, 18), Decimal(\'1676.14\'), \'Atelier graphique\'), (datetime.date(2004, 12, 17), Decimal(\'14191.12\'), \'Signal Gift Stores\'), (datetime.date(2003, 6, 6), Decimal(\'32641.98\'), \'Signal Gift Stores\'), (datetime.date(2004, 8, 20), Decimal(\'33347.88\'), \'Signal Gift Stores\'), (datetime.date(2003, 5, 20), Decimal(\'45864.03\'), \'Australian Collectors, Co.\'), (datetime.date(2004, 12, 15), Decimal(\'82261.22\'), \'Australian Collectors, Co.\'), (datetime.date(2003, 5, 31), Decimal(\'7565.08\'), \'Australian Collectors, Co.\'), (datetime.date(2004, 3, 10), Decimal(\'44894.74\'), \'Australian Collectors, Co.\'), (datetime.date(2004, 11, 14), Decimal(\'19501.82\'), \'La Rochelle Gifts\'), (datetime.date(2004, 8, 8), Decimal(\'47924.19\'), \'La Rochelle Gifts\'), (datetime.date(2005, 2, 22), Decimal(\'49523.67\

In [19]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "give me all the payment detail with customer name"})

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'NULLS LAST\' at line 1")\n[SQL: SELECT payments.customerNumber, customers.customerName, payments.paymentDate, payments.amount FROM payments JOIN customers ON payments.customerNumber = customers.customerNumber ORDER BY payments.customerNumber NULLS LAST;]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [14]:
import csv
# Specify the CSV file path
csv_file_path = 'output.csv'

# Extract column names from the query (if available, otherwise define them manually)
column_names = ['paymentDate', 'amount', 'customerName']

# Write the data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(column_names)  # Write the header row
    writer.writerows(x)  # Write the data rows

print(f'Data has been written to {csv_file_path}')

Data has been written to output.csv


In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

"[(Decimal('95.70'),)]"

In [15]:
execute_query.invoke(query)

str

In [6]:
chain = write_query | execute_query
chain.invoke({"question":"how many orders are there"})

'[(326,)]'

In [7]:
pip install -U langsmith

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]="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"

In [9]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many Orders are there"})

'326'

# FewShortPrompt

In [10]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
        'input': "what is price of 1968 Ford Mustang",
        "query": "SELECT buyPrice`, MSRP FROM products WHERE productName = '1968 Ford Mustang' LIMIT 1;"
    }
]

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input", "top_k"],
    input_variables=["input"],
)

print(few_shot_prompt.format(input="How many products are there?", top_k=3, table_info=""))

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: what is price of 1968 Ford Mustang
SQLQuery:
AI: SELECT buyPrice`, MSRP FROM products WHERE productName = '1968 Ford Mustang' LIMIT 1;


# Dynamic FewshotPrompt

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\royel\anaconda3\envs\medical_chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\royel\anaconda3\envs\medical_chatbot\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\royel\anaconda3\envs\medical_chatbot\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

vectorstore = Chroma()
vectorstore.delete_collection()

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    vectorstore,
    k=2,
    input_keys=["input"],
)

example_selector.select_examples({"input":"who is highest paying customer"})

[{'input': 'Get the highest payment amount made by any customer.',
  'query': 'SELECT MAX(amount) FROM payments;'},
 {'input': 'List all customers in France with a credit limit over 20,000.',
  'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"}]

In [14]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector, 
    input_variables=["input","top_k"],
)

print(few_shot_prompt.format(input="How many products are there?",top_k=3,table_info=""))

Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


# Customizing Prompts

In [15]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a MySQL expert. Given an input question, Create a synthatically correct Mysql query to run. Unless otherwise specificed. \n\nOnly use the following tables: {table_info}\n\n below are a number of examples of question and their corresponding SQL queries."),
        few_shot_prompt,
        ("human","{input}")
    ]
)
print(final_prompt.format(input="how many products are there?",table_info="some table info"))

System: You are a MySQL expert. Given an input question, Create a synthatically correct Mysql query to run. Unless otherwise specificed. 

Only use the following tables: some table info

 below are a number of examples of question and their corresponding SQL queries.
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: how many products are there?


In [16]:
generate_query = create_sql_query_chain(llm,db,final_prompt)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many customers with credit limit more than 50000"})

'1670711'

# Tabele Details

In [17]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [18]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("table_info.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)

Table Name:productlines
Table Description:Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines.

Table Name:products
Table Description:Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP. Linked to the productlines table.

Table Name:offices
Table Description:Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.

Table Name:employees
Table Description:Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reportsTo attribute.

Table Name:customers
Table Description:Captures data on cus

In [19]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer and their order count"})
tables


c:\Users\royel\anaconda3\envs\medical_chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: https://python.langchain.com/docs/modules/model_io/chat/structured_output/Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.https://python.langchain.com/docs/use_cases/extraction/.with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:https://github.com/langchain-ai/langchain/discussions/18154
  warn_deprecated(


[]

In [20]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer and their order count"})


[]

In [21]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| answer
)
chain.invoke({"question": "How many cutomers with order count more than 5"})


'2019071'

# Chatbots with Memory 

In [22]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

In [23]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


In [24]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| answer
)


In [25]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})

In [26]:
history.add_user_message(question)
history.add_ai_message(response)
history.messages

[HumanMessage(content='How many cutomers with order count more than 5'),
 AIMessage(content='0')]

In [29]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response

'---------------------------------------------------------------------------'

: 